GBA1 - Single gene analysis in AMP-PD WGS data (EUR population)

Notebook Overview

1. Loading Python libraries 

2. Installing packages

3. copy data

4. Create a covariate file with AMP-PD data

5. Annotation of the gene GBA1

6. Burden test(SkatO, Skat, cmc,zeggini,mb,fp,cmcWald)

7. Save out results

Loading Python libraries

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics

import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

/tmp/ipykernel_2369/2693492717.py:22: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Defining functions

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

In [4]:
ls /home/jupyter/tools/

annovar/               plink2*                          rvtests/
annovar.latest.tar.gz  plink2_linux_x86_64_latest.zip   toy.map
LICENSE                plink_linux_x86_64_20190304.zip  toy.ped
plink*                 prettify*


In [4]:
%%bash

# chmod plink 1.9 to make sure you have permission to run the program
chmod u+x /home/jupyter/tools/plink
# chmod plink 2.0 to make sure you have permission to run the program
chmod u+x /home/jupyter/tools/plink2
# Change permissions
chmod 777 /home/jupyter/tools/rvtests/

Copy data from AMP-PD bucket to workspace

In [6]:
WORK_DIR = "/home/jupyter/"

In [7]:
#Copy data
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_WGS_RELEASE_PLINK_PATH}/pfiles/chr1.* {WORK_DIR}')

Executing: gsutil -u terra-ddf4a6f6 -m cp -r gs://amp-pd-genomics/releases/2023_v4release_1027/wgs-WB-DWGS/plink/pfiles/chr1.* /home/jupyter/


Copying gs://amp-pd-genomics/releases/2023_v4release_1027/wgs-WB-DWGS/plink/pfiles/chr1.pgen...
Copying gs://amp-pd-genomics/releases/2023_v4release_1027/wgs-WB-DWGS/plink/pfiles/chr1.psam...
Copying gs://amp-pd-genomics/releases/2023_v4release_1027/wgs-WB-DWGS/plink/pfiles/chr1.pvar...
/ [3/3 files][ 18.7 GiB/ 18.7 GiB] 100% Done  39.3 MiB/s ETA 00:00:00           
Operation completed over 3 objects/18.7 GiB.                                     


In [9]:
! ls /home/jupyter/GBA_AMPPD

chr1_updated_formatted.log   chr1_updated.log	COVS.txt
chr1_updated_formatted.pgen  chr1_updated.pgen	GBA.log
chr1_updated_formatted.psam  chr1_updated.psam
chr1_updated_formatted.pvar  chr1_updated.pvar


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r /home/jupyter/COVS.txt /home/jupyter/GBA_AMPPD/COVS.txt')

Update sex and pheno info

In [ ]:
# Update sex and phenotype in plink files, using the COVS.txt file that we made
#-update-sex and column number tells plink where to look for the sex information in the file

WORK_DIR = "/home/jupyter/"

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr1 \
--update-sex {WORK_DIR}/COVS.txt col-num=5 \
--pheno {WORK_DIR}/COVS.txt \
--pheno-name PHENO \
--make-pgen \
--out {WORK_DIR}/GBA_AMPPD/chr1_updated

In [10]:
%%bash

head /home/jupyter/GBA_AMPPD/chr1_updated.psam

#FID	IID	SEX	PHENO
HB-PD_INVAB109VHC	HB-PD_INVAB109VHC	2	1
HB-PD_INVAB289LG3	HB-PD_INVAB289LG3	2	1
HB-PD_INVAC488AAF	HB-PD_INVAC488AAF	1	1
HB-PD_INVAE296YP8	HB-PD_INVAE296YP8	2	1
HB-PD_INVAJ549VWD	HB-PD_INVAJ549VWD	2	1
HB-PD_INVAK106DV5	HB-PD_INVAK106DV5	1	1
HB-PD_INVAK293NGR	HB-PD_INVAK293NGR	1	1
HB-PD_INVAM835KNR	HB-PD_INVAM835KNR	2	1
HB-PD_INVAM864YC9	HB-PD_INVAM864YC9	2	1


In [11]:
! grep "BF-1001" /home/jupyter/GBA_AMPPD/chr1_updated.psam

BF-1001	BF-1001	1	1


Extract and annotate the gene Extract the region using PLINK Extract GBA gene GBA coordinates: Chromosome 1:155234452-1:155244699(GRCh38/hg38)

In [ ]:
## update the variant IDs

! /home/jupyter/tools/plink2 \
--pfile /home/jupyter/GBA_AMPPD/chr1_updated \
--fa /home/jupyter/hg38.fa \
--memory 25000 \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/GBA_AMPPD/chr1_updated_formatted

In [12]:
## extract region using plink and make pgen files
WORK_DIR = "/home/jupyter/"

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/GBA_AMPPD/chr1_updated_formatted \
--chr 1 \
--from-bp 155184452 \
--to-bp 155294699 \
--max-alleles 2 \
--make-pgen \
--out {WORK_DIR}/GBA_AMPPD/GBA

PLINK v2.00a6LM 64-bit Intel (17 Jun 2024)     www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter//GBA_AMPPD/GBA.log.
Options in effect:
  --chr 1
  --from-bp 155184452
  --make-pgen
  --max-alleles 2
  --out /home/jupyter//GBA_AMPPD/GBA
  --pfile /home/jupyter//GBA_AMPPD/chr1_updated_formatted
  --to-bp 155294699

Start time: Wed Nov 20 13:32:49 2024
30093 MiB RAM detected, ~28416 available; reserving 15046 MiB for main
workspace.
Using up to 8 compute threads.
10418 samples (4462 females, 5627 males, 329 ambiguous; 10418 founders) loaded
from /home/jupyter//GBA_AMPPD/chr1_updated_formatted.psam.
11357160 out of 12349094 variants loaded from
/home/jupyter//GBA_AMPPD/chr1_updated_formatted.pvar.
1 binary phenotype loaded (3188 cases, 4014 controls).
5190 variants remaining after main filters.
Writing /home/jupyter//GBA_AMPPD/GBA.psam ... done.
Writing /home/jupyter//GBA_AMPPD/GBA.pvar ... 1010111112121

In [13]:
## extract region using plink and make binary files
WORK_DIR = "/home/jupyter/"

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/GBA_AMPPD/chr1_updated_formatted \
--chr 1 \
--from-bp 155184452 \
--to-bp 155294699 \
--max-alleles 2 \
--make-bed \
--out {WORK_DIR}/GBA_AMPPD/GBA

PLINK v2.00a6LM 64-bit Intel (17 Jun 2024)     www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter//GBA_AMPPD/GBA.log.
Options in effect:
  --chr 1
  --from-bp 155184452
  --make-bed
  --max-alleles 2
  --out /home/jupyter//GBA_AMPPD/GBA
  --pfile /home/jupyter//GBA_AMPPD/chr1_updated_formatted
  --to-bp 155294699

Start time: Wed Nov 20 13:41:36 2024
30093 MiB RAM detected, ~28438 available; reserving 15046 MiB for main
workspace.
Using up to 8 compute threads.
10418 samples (4462 females, 5627 males, 329 ambiguous; 10418 founders) loaded
from /home/jupyter//GBA_AMPPD/chr1_updated_formatted.psam.
11357160 out of 12349094 variants loaded from
/home/jupyter//GBA_AMPPD/chr1_updated_formatted.pvar.
1 binary phenotype loaded (3188 cases, 4014 controls).
5190 variants remaining after main filters.
Writing /home/jupyter//GBA_AMPPD/GBA.fam ... done.
Writing /home/jupyter//GBA_AMPPD/GBA.bim ... done.
Writing /h

In [14]:
# Visualize bim file
! head {WORK_DIR}/GBA_AMPPD/GBA.bim

1	chr1:155184542:ACTGGGCATGAT:A	0	155184542	A	ACTGGGCATGAT
1	chr1:155184559:G:T	0	155184559	T	G
1	chr1:155184604:C:T	0	155184604	T	C
1	chr1:155184621:G:A	0	155184621	A	G
1	chr1:155184622:C:A	0	155184622	A	C
1	chr1:155184639:C:T	0	155184639	T	C
1	chr1:155184670:G:A	0	155184670	A	G
1	chr1:155184680:A:T	0	155184680	T	A
1	chr1:155184709:C:A	0	155184709	A	C
1	chr1:155184712:C:T	0	155184712	T	C


Turn binary files into VCF

In [16]:
## Turn binary files into VCF
WORK_DIR = "/home/jupyter/"

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/GBA_AMPPD/GBA \
--recode vcf id-paste=iid \
--mac 2 \
--out {WORK_DIR}/GBA_AMPPD/GBA

PLINK v2.00a6LM 64-bit Intel (17 Jun 2024)     www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter//GBA_AMPPD/GBA.log.
Options in effect:
  --bfile /home/jupyter//GBA_AMPPD/GBA
  --export vcf id-paste=iid
  --mac 2
  --out /home/jupyter//GBA_AMPPD/GBA

Start time: Wed Nov 20 13:45:12 2024
30093 MiB RAM detected, ~28443 available; reserving 15046 MiB for main
workspace.
Using up to 8 compute threads.
10418 samples (4462 females, 5627 males, 329 ambiguous; 10418 founders) loaded
from /home/jupyter//GBA_AMPPD/GBA.fam.
5190 variants loaded from /home/jupyter//GBA_AMPPD/GBA.bim.
1 binary phenotype loaded (3188 cases, 4014 controls).
Calculating allele frequencies... done.
3174 variants removed due to allele frequency threshold(s)
(--maf/--max-maf/--mac/--max-mac).
2016 variants remaining after main filters.
--export vcf to /home/jupyter//GBA_AMPPD/GBA.vcf ... 1010111112121313141415151616171718181919202021212

In [17]:
### Bgzip and Tabix (zip and index the file)

! bgzip -f {WORK_DIR}/GBA_AMPPD/GBA.vcf
! tabix -f -p vcf {WORK_DIR}/GBA_AMPPD/GBA.vcf.gz

Annotate using ANNOVAR

In [18]:
## annotate using ANNOVAR
! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/GBA_AMPPD/GBA.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out {WORK_DIR}/GBA_AMPPD/GBA.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 /home/jupyter//GBA_AMPPD/GBA.vcf.gz > /home/jupyter//GBA_AMPPD/GBA.annovar.avinput>
NOTICE: Finished reading 2023 lines from VCF file
NOTICE: A total of 2016 locus in VCF file passed QC threshold, representing 1867 SNPs (1321 transitions and 546 transversions) and 149 indels/substitutions
NOTICE: Finished writing allele frequencies based on 19450406 SNP genotypes (13762178 transitions and 5688228 transversions) and 1552282 indels/substitutions for 10418 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl /home/jupyter//GBA_AMPPD/GBA.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile /home/jupyter//GBA_AMPPD/GBA.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refG

In [19]:
# Read in ANNOVAR multianno file
gene = pd.read_csv(f'{WORK_DIR}/GBA_AMPPD/GBA.annovar.hg38_multianno.txt', sep = '\t')
display(gene)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo10421,Otherinfo10422,Otherinfo10423,Otherinfo10424,Otherinfo10425,Otherinfo10426,Otherinfo10427,Otherinfo10428,Otherinfo10429,Otherinfo10430
0,1,155184604,155184604,C,T,UTR3,TRIM46,NM_001282378:c.*414C>T;NM_025058:c.*414C>T;NM_...,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,1,155184622,155184622,C,A,UTR3,TRIM46,NM_001282378:c.*432C>A;NM_025058:c.*432C>A;NM_...,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,1,155184763,155184763,G,T,UTR3,TRIM46,NM_001282378:c.*573G>T;NM_025058:c.*573G>T;NM_...,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,1,155184969,155184969,G,A,UTR3,TRIM46,NM_001282378:c.*779G>A;NM_025058:c.*779G>A;NM_...,.,.,...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,1,155185007,155185007,G,A,downstream,MUC1;TRIM46,dist=817,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,1,155294470,155294470,C,T,intronic,PKLR,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2012,1,155294538,155294538,C,G,exonic,PKLR,.,synonymous SNV,"PKLR:NM_000298:exon6:c.G909C:p.P303P,PKLR:NM_1...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2013,1,155294560,155294560,G,T,exonic,PKLR,.,nonsynonymous SNV,"PKLR:NM_000298:exon6:c.C887A:p.A296D,PKLR:NM_1...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2014,1,155294618,155294618,C,T,exonic,PKLR,.,nonsynonymous SNV,"PKLR:NM_000298:exon6:c.G829A:p.E277K,PKLR:NM_1...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [20]:
results = [] 


intronic = gene[gene['Func.refGene']== 'intronic']
upstream = gene[gene['Func.refGene']== 'upstream']
downstream = gene[gene['Func.refGene']== 'downstream']
utr5 = gene[gene['Func.refGene']== 'UTR5']
utr3 = gene[gene['Func.refGene']== 'UTR3']
splicing = gene[gene['Func.refGene']== 'splicing']
exonic = gene[gene['Func.refGene']== 'exonic']
stopgain = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'stopgain')]
stoploss = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'stoploss')]
startloss = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'startloss')]
frameshift_deletion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'frameshift deletion')]
frameshift_insertion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'frameshift insertion')]
nonframeshift_deletion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonframeshift deletion')]
nonframeshift_insertion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonframeshift insertion')]
coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'synonymous SNV')]
lof = exonic[(gene['ExonicFunc.refGene'] == 'stopgain') | (gene['ExonicFunc.refGene'] == 'stoploss') | (gene['ExonicFunc.refGene'] == 'frameshift deletion') | (gene['ExonicFunc.refGene'] == 'frameshift insertion') | (gene['ExonicFunc.refGene'] == 'startloss')]
nonsynonymous_lof = pd.concat([coding_nonsynonymous, lof])

print('Total variants: ', len(gene))
print("exonic: ", len(exonic))
print("Intronic: ", len(intronic))
print("Upstream: ", len(upstream))
print("Downstream: ", len(downstream))
print('UTR3: ', len(utr3))
print('UTR5: ', len(utr5))
print("Splicing: ", len(splicing))
print("Total exonic: ", len(exonic))
print("Stopgain: ", len(stopgain))
print("Stoploss: ", len(stoploss))
print("Startloss: ", len(startloss))
print("Frameshift deletion: ", len(frameshift_deletion))
print("Frameshift insertion: ", len(frameshift_insertion))
print("Non-frameshift insertion: ", len(nonframeshift_insertion))
print("Non-frameshift deletion: ", len(nonframeshift_deletion))
print('Synonymous: ', len(coding_synonymous))
print("Nonsynonymous: ", len(coding_nonsynonymous))
print("nonsynonymous_lof: ", len(nonsynonymous_lof))
results.append((gene, intronic, upstream, downstream, utr3, utr5, splicing,exonic,stopgain,stoploss,startloss, frameshift_deletion,frameshift_insertion,nonframeshift_deletion,nonframeshift_insertion,coding_synonymous, coding_nonsynonymous, nonsynonymous_lof))
print('\n')

Total variants:  2016
exonic:  267
Intronic:  1026
Upstream:  111
Downstream:  49
UTR3:  75
UTR5:  42
Splicing:  3
Total exonic:  267
Stopgain:  3
Stoploss:  0
Startloss:  0
Frameshift deletion:  5
Frameshift insertion:  4
Non-frameshift insertion:  1
Non-frameshift deletion:  1
Synonymous:  77
Nonsynonymous:  176
nonsynonymous_lof:  188




/tmp/ipykernel_84/996593315.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lof = exonic[(gene['ExonicFunc.refGene'] == 'stopgain') | (gene['ExonicFunc.refGene'] == 'stoploss') | (gene['ExonicFunc.refGene'] == 'frameshift deletion') | (gene['ExonicFunc.refGene'] == 'frameshift insertion') | (gene['ExonicFunc.refGene'] == 'startloss')]


In [21]:
# Save in PLINK format
variants_toKeep = exonic[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep.to_csv(f'{WORK_DIR}/GBA_AMPPD/GBA.exonic.variantstoKeep.txt', sep="\t", index=False, header=False)

In [22]:
# Save in PLINK format
variants_toKeep = nonsynonymous_lof[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep.to_csv(f'{WORK_DIR}/GBA_AMPPD/GBA.nonsynonymous_lof.variantstoKeep.txt', sep="\t", index=False, header=False)

Burden Analyses using RVTests

In [23]:
#Loop over 2 variant classes, extract variants and run rvtests
##GBA
variant_classes = ['exonic', 'nonsynonymous_lof']

WORK_DIR = "/home/jupyter/"

for variant_class in variant_classes:
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running plink to extract {variant_class} variants for ancestry: AMPPD')
    
    #Extract relevant variants
    ! /home/jupyter/tools/plink2 \
    --pfile {WORK_DIR}/GBA_AMPPD/GBA \
    --extract range {WORK_DIR}/GBA_AMPPD/GBA.{variant_class}.variantstoKeep.txt \
    --recode vcf-iid \
    --out {WORK_DIR}/GBA_AMPPD/GBA.{variant_class}
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running bgzip and tabix for {variant_class} variants for ancestry: AMPPD')
        
    ## Bgzip and Tabix (zip and index the file)
    ! bgzip -f {WORK_DIR}/GBA_AMPPD/GBA.{variant_class}.vcf
    ! tabix -f -p vcf {WORK_DIR}/GBA_AMPPD/GBA.{variant_class}.vcf.gz

Running plink to extract exonic variants for ancestry: AMPPD
PLINK v2.00a6LM 64-bit Intel (17 Jun 2024)     www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter//GBA_AMPPD/GBA.exonic.log.
Options in effect:
  --export vcf-iid
  --extract range /home/jupyter//GBA_AMPPD/GBA.exonic.variantstoKeep.txt
  --out /home/jupyter//GBA_AMPPD/GBA.exonic
  --pfile /home/jupyter//GBA_AMPPD/GBA

Start time: Wed Nov 20 13:58:11 2024
Note: --export 'vcf-iid' modifier is deprecated.  Use 'vcf' + 'id-paste=iid'.
30093 MiB RAM detected, ~27911 available; reserving 15046 MiB for main
workspace.
Using up to 8 compute threads.
10418 samples (4462 females, 5627 males, 329 ambiguous; 10418 founders) loaded
from /home/jupyter//GBA_AMPPD/GBA.psam.
5190 variants loaded from /home/jupyter//GBA_AMPPD/GBA.pvar.
1 binary phenotype loaded (3188 cases, 4014 controls).
--extract bed1: 4929 variants excluded.
261 variants remaining after ma

In [ ]:
WORK_DIR = "/home/jupyter/"

variant_classes = ['exonic', 'nonsynonymous_lof']

for variant_class in variant_classes:
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running RVtests for {variant_class} variants for ancestry: AMPPD')
        
     ## RVtests with covariates 
    #Make sure the pheno and covariate file starts with the first 5 columsn: fid, iid, fatid, matid, sex
    #The pheno-name flag only works when the pheno/covar file is structured properly
    ! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
    --out {WORK_DIR}/GBA_AMPPD/GBA.burden.{variant_class} \
    --kernel skato \
    --inVcf {WORK_DIR}/GBA_AMPPD/GBA.{variant_class}.vcf.gz \
    --pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --pheno-name PHENO \
    --gene GBA \
    --geneFile {WORK_DIR}/refFlat_HG38.txt \
    --covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
    --freqUpper 0.01

In [7]:
## look at results in exonic
! cat {WORK_DIR}/GBA_AMPPD/GBA.burden.exonic.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	42	29	95921.2	1	0.00861309


In [8]:
## look at results in nonsynonymous_lof
! cat {WORK_DIR}/GBA_AMPPD/GBA.burden.nonsynonymous_lof.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	62217	1	0.0202248


In [ ]:
# burden test in AMPPD ancestry
#Make sure the pheno and covariate file starts with the first 5 columsn: fid, iid, fatid, matid, sex
        #The pheno-name flag only works when the pheno/covar file is structured properly
! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
--out {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.exonic \
--burden cmc,zeggini,mb,fp,cmcWald \
--inVcf {WORK_DIR}/GBA_AMPPD/GBA.exonic.vcf.gz \
--pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
--pheno-name PHENO \
--gene GBA \
--geneFile {WORK_DIR}/refFlat_HG38.txt \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--freqUpper 0.01

In [10]:
# see bureden test in exonic in AMPPD
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.exonic.CMCWald.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	NonRefSite	Beta	SE	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	42	29	203	0.415272	0.154824	0.00731365
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	42	29	203	-0.640712	0.0613669	1.61654e-25
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2

In [11]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.exonic.CMC.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	NonRefSite	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	42	29	203	0.00705455


In [12]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.exonic.Fp.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	42	29	0.00144637


In [31]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.exonic.MadsonBrowning.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	NumPerm	ActualPerm	Stat	NumGreater	NumEqual	PermPvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	7202	41	34	10000	10000	18.4115	4	0	0.0004


In [13]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.exonic.Zeggini.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	42	29	0.01077


In [ ]:
# burden test in AMPPD ancestry
#Make sure the pheno and covariate file starts with the first 5 columsn: fid, iid, fatid, matid, sex
        #The pheno-name flag only works when the pheno/covar file is structured properly
! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
--out {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof \
--burden cmc,zeggini,mb,fp,cmcWald \
--inVcf {WORK_DIR}/GBA_AMPPD/GBA.nonsynonymous_lof.vcf.gz \
--pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
--pheno-name PHENO \
--gene GBA \
--geneFile {WORK_DIR}/refFlat_HG38.txt \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--freqUpper 0.01

In [15]:
# see bureden test in nonsynonymous_lof in AMPPD
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof.CMCWald.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	NonRefSite	Beta	SE	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	173	0.419539	0.167773	0.0123968
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	173	-0.640728	0.0613631	1.60119e-25
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-25

In [16]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof.CMC.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	NonRefSite	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	173	0.012


In [17]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof.Fp.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	0.00465715


In [18]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof.MadsonBrowning.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	NumPerm	ActualPerm	Stat	NumGreater	NumEqual	PermPvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	10000	10000	10.6587	46	0	0.0046


In [19]:
! cat {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof.Zeggini.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Pvalue
GBA	1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-2561,1_GL383519v1_alt:0-3358,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3426,1_GL383519v1_alt:39474-46272,1_GL383519v1_alt:0-3339,1_GL383519v1_alt:39474-49650,1_GL383519v1_alt:0-3426,1:155234451-155241249,1:155234451-155241249,1:155234451-155244627,1:155234451-155244627,1:155234451-155244627	5086	28	20	0.0122216


In [ ]:
# run single variant burden analysis
# single variant analysis
! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
--inVcf {WORK_DIR}/GBA_AMPPD/GBA.exonic.vcf.gz \
--single score \
--pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
--pheno-name PHENO \
--geneFile {WORK_DIR}/refFlat_HG38.txt \
--gene GBA \
--out {WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTESTSV.exonic \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--freqUpper 0.01

In [21]:
SV_AMPPDexonic = pd.read_csv(f'{WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc', sep = '\t')

In [22]:
# Drop duplicate rows
SV_AMPPDexonic = SV_AMPPDexonic.drop_duplicates()
display(SV_AMPPDexonic)

,Gene,CHROM,POS,REF,ALT,N_INFORMATIVE,AF,U,V,STAT,DIRECTION,EFFECT,SE,PVALUE
0,GBA,1,155235196,G,A,5086,0.000197,0.644930,0.429459,0.968507,+,1.501730,1.525950,0.325053
1,GBA,1,155235203,C,G,5086,0.002064,0.568009,4.718310,0.068379,+,0.120384,0.460370,0.793711
2,GBA,1,155235206,G,A,5086,0.000098,0.632278,0.232255,1.721280,+,2.722340,2.075000,0.189529
3,GBA,1,155235211,C,G,5086,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GBA,1,155235217,C,G,5086,0.001475,0.025041,3.522080,0.000178,+,0.007110,0.532844,0.989354
5,GBA,1,155235227,G,A,5086,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GBA,1,155235245,T,C,5086,0.000197,-0.951925,0.389797,2.324700,-,-2.442100,1.601700,0.127335
7,GBA,1,155235256,C,T,5086,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GBA,1,155235269,G,C,5086,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,GBA,1,155235727,C,G,5086,0.000295,-0.524750,0.739292,0.372468,-,-0.709801,1.163030,0.541662


In [23]:
SV_AMPPDexonic.to_csv(f'{WORK_DIR}/GBA_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv')

In [43]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r /home/jupyter/CTSB_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv /home/jupyter/GBA_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv')

Executing: gsutil -mu terra-ddf4a6f6 cp -r /home/jupyter/CTSB_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv /home/jupyter/GBA_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv


Copying file:///home/jupyter/CTSB_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv...
/ [1/1 files][  3.4 KiB/  3.4 KiB] 100% Done                                    
Operation completed over 1 objects/3.4 KiB.                                      


In [46]:
! ls /home/jupyter/GBA_AMPPD

chr1_updated_formatted.log
chr1_updated_formatted.pgen
chr1_updated_formatted.psam
chr1_updated_formatted.pvar
chr1_updated.log
chr1_updated.pgen
chr1_updated.psam
chr1_updated.pvar
COVS.txt
GBA_AMPPD.burdenTEST.exonic.CMC.assoc
GBA_AMPPD.burdenTEST.exonic.CMCWald.assoc
GBA_AMPPD.burdenTEST.exonic.Fp.assoc
GBA_AMPPD.burdenTEST.exonic.log
GBA_AMPPD.burdenTEST.exonic.MadsonBrowning.assoc
GBA_AMPPD.burdenTEST.exonic.Zeggini.assoc
GBA_AMPPD.burdenTESTSV.exonic.log
GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc
GBA_AMPPD.burdenTESTSV.exonic.SingleScore.assoc.csv
GBA.annovar.avinput
GBA.annovar.hg38_multianno.txt
GBA.annovar.hg38_multianno.vcf
GBA.bed
GBA.bim
GBA.burden.exonic.log
GBA.burden.exonic.SkatO.assoc
GBA.burden.nonsynonymous_lof.log
GBA.burden.nonsynonymous_lof.SkatO.assoc
GBA.exonic.log
GBA.exonic.variantstoKeep.txt
GBA.exonic.vcf.gz
GBA.exonic.vcf.gz.tbi
GBA.fam
GBA.log
GBA.nonsynonymous_lof.log
GBA.nonsynonymous_lof.variantstoKeep.txt
GBA.nonsynonymous_lof.vcf.gz
GBA.nonsynonym

In [24]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r /home/jupyter/GBA_AMPPD/ {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-ddf4a6f6 cp -r /home/jupyter/GBA_AMPPD/ gs://fc-49394ff4-0f5f-49e6-a30a-cf07b1dba334


Copying file:///home/jupyter/GBA_AMPPD/GBA.nonsynonymous_lof.vcf.gz.tbi [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/GBA_AMPPD.burdenTEST.nonsynonymous_lof.MadsonBrowning.assoc [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/GBA_AMPPD.burdenTESTSV.exonic.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/GBA.exonic.variantstoKeep.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/GBA_AMPPD/GBA.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/GBA.burden.nonsynonymous_lof.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/chr1_updated_formatted.psam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/COVS.txt [Content-Type=text/plain]...    
Copying file:///home/jupyter/GBA_AMPPD/GBA.pvar [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/GBA_AMPPD/GBA